<a href="https://colab.research.google.com/github/VitaliyShein/Clusterizer/blob/main/ClusterizerWithLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [168]:
%%python -m pip install pip<24.1

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [169]:
%pip install textract
import textract

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [170]:
%pip install langdetect

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [171]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
import shutil
import os
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity

In [172]:
import logging

# Настройка логирования
# logging.basicConfig(
#     level=logging.DEBUG,
#     filemode='w',
#     handler=logging.StreamHandler()
# )
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [173]:
from google.colab import drive
logger.info("Монтирование Google Drive")
drive.mount('/content/drive', force_remount=True)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

Mounted at /content/drive


In [174]:
file_handler = logging.FileHandler('/content/drive/MyDrive/Project0/clusterizer.log', mode='w')
formatter = logging.Formatter("[%(name)s] [%(asctime)s] [%(levelname)s]: %(message)s")
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)

In [175]:
"""Загрузка BERT"""

from transformers import BertTokenizer, pipeline
import torch

# Для английского
en_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
logger.info("Загружен суммаризатор для английского языка")

# Для русского
ru_summarizer = pipeline(
    "summarization",
    model="IlyaGusev/rut5_base_sum_gazeta",
    tokenizer="IlyaGusev/mbart_ru_sum_gazeta"
)
logger.info("Загружен суммаризатор для русского языка")

def summarize_text(text, summarizer):
    """Генерация краткого описания"""
    try:
        logger.info(f"Суммаризация текста длиной {len(text)} символов")
        summary = summarizer(
          text[:2000] if len(text) > 20 else text,
          max_length=15,
          min_length=3,
          length_penalty=0.2  # Делаем суммаризацию короче
        )
        logger.debug(f"Результат суммаризации: {summary[0]['summary_text']}")
        return summary[0]['summary_text']
    except Exception as e:
        logger.error(f"Ошибка суммаризации: {e}")
        return 0

Device set to use cpu
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line

In [176]:
def find_closest_to_centroids(embeddings, centroids, labels=None, texts=None):
    """
    Находит ближайшие примеры к каждому центроиду кластера

    Args:
        embeddings (np.array): Массив эмбеддингов (n_samples, n_features)
        centroids (np.array): Центроиды кластеров (n_clusters, n_features)
        labels (np.array): Метки кластеров для каждого примера (опционально)
        texts (list): Список текстов (опционально)

    Returns:
        dict: {
            'closest_indices': индексы ближайших примеров,
            'closest_embeddings': эмбеддинги ближайших примеров,
            'similarity_scores': косинусные сходства,
            'closest_texts': ближайшие тексты (если texts переданы)
        }

    Raises:
        ValueError: При некорректных входных данных
    """
    logger.info("Поиск ближайших примеров к центроидам")
    logger.debug(f"Входные данные: embeddings {embeddings}, centroids {centroids}")

    # Валидация входных данных
    if embeddings is None or centroids is None:
        logger.error("Не переданы embeddings или centroids")
        raise ValueError("Embeddings and centroids cannot be None")

    if len(embeddings) == 0 or len(centroids) == 0:
        logger.error("Пустые массивы embeddings или centroids")
        raise ValueError("Empty arrays provided")


    # # Валидация размерностей
    # if embeddings.shape[0] != 2:
    #     logger.error(f"Ожидается 2D массив embeddings, получено {embeddings.ndim}D")
    #     raise ValueError("Embeddings must be 2D array")

    # if centroids.shape[0] != 2:
    #     logger.error(f"Ожидается 2D массив centroids, получено {centroids.ndim}D")
    #     raise ValueError("Centroids must be 2D array")

    if embeddings.shape[1] != centroids.shape[1]:
        logger.error(f"Несовпадение размерностей: embeddings {embeddings.shape[1]}, centroids {centroids.shape[1]}")
        raise ValueError("Dimension mismatch between embeddings and centroids")

        # Преобразование в numpy array если нужно
    if not isinstance(embeddings, np.ndarray):
        logger.debug("Преобразование embeddings в numpy array")
        embeddings = np.array(embeddings)

    if not isinstance(centroids, np.ndarray):
        logger.debug("Преобразование centroids в numpy array")
        centroids = np.array(centroids)

    n_clusters = len(centroids)
    closest_indices = []
    closest_embeddings = []
    similarity_scores = []

    # Поиск ближайших примеров для каждого центроида
    for i, centroid in enumerate(centroids):
        try:
            logger.debug(f"Обработка центроида {i}")

            # Если есть метки, ищем только в соответствующем кластере
            if labels is not None:
                cluster_mask = (labels == i)
                cluster_embeddings = embeddings[cluster_mask]
                original_indices = np.where(cluster_mask)[0]

                nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(doc_embeddings)
                distances, indices = nbrs.kneighbors([centroid])

                logger.debug(f"Кластер {i} содержит {len(cluster_embeddings)} примеров")
            else:
                cluster_embeddings = embeddings
                original_indices = np.arange(len(embeddings))
                logger.debug("Метки кластеров не предоставлены, поиск по всем примерам")

            if len(cluster_embeddings) == 0:
                logger.warning(f"Кластер {i} пуст!")
                closest_indices.append(-1)
                closest_embeddings.append(None)
                similarity_scores.append(0)
                continue

            # Вычисляем косинусное сходство
            similarities = cosine_similarity(cluster_embeddings, centroid.reshape(1, -1)).flatten()

            # Находим ближайший пример
            closest_idx_in_cluster = np.argmax(similarities)
            closest_idx = original_indices[closest_idx_in_cluster]
            similarity = similarities[closest_idx_in_cluster]

            closest_indices.append(closest_idx)
            closest_embeddings.append(embeddings[closest_idx])
            similarity_scores.append(similarity)

            logger.debug(f"Центроид {i}: ближайший пример idx={closest_idx}, сходство={similarity:.3f}")

        except Exception as e:
            logger.error(f"Ошибка при обработке центроида {i}: {str(e)}")
            raise

    # Формируем результат
    result = {
        'closest_indices': closest_indices,
        'closest_embeddings': closest_embeddings,
        'similarity_scores': similarity_scores
    }

    # Добавляем тексты, если они были переданы
    if texts is not None:
        if len(texts) != len(embeddings):
            logger.error(f"Несоответствие количества текстов ({len(texts)}) и эмбеддингов ({len(embeddings)})")
            raise ValueError("Texts and embeddings length mismatch")

        # result['closest_texts'] = [texts[i] if i != -1 else None for i in closest_indices]
        closest_texts = [texts[i] if i != -1 else None for i in closest_indices]
        logger.debug("Добавлены ближайшие тексты в результат")

    logger.info(f"Успешно найдены ближайшие примеры для {n_clusters} центроидов")
    return closest_texts

In [177]:
def predict_cluster(text, kmeans_model):
    """
    Предсказывает кластер для нового текста

    Args:
        text (str): Текст для классификации
        kmeans_model: Обученная модель кластеризации

    Returns:
        int: Номер кластера (0, 1, 2, ...)
    """
    logger.info(f"Предсказание кластера для текста длиной {len(text)} символов")
    embedding = get_ONE_embedding(text)
    cluster = kmeans_model.predict(embedding)
    logger.info(f"Предсказан кластер {cluster[0]} для текста")
    return cluster[0]

In [178]:
def train_cluster_model(embeddings=[], n_clusters=3, random_state=42):
    """
    Обучает модель кластеризации на эмбеддингах

    Args:
        embeddings (np.array): Массив эмбеддингов
        n_clusters (int): Количество кластеров
        random_state (int): Для воспроизводимости

    Returns:
        KMeans: Обученная модель K-Means
    """
    if embeddings is None:
        logger.error("Не переданы эмбеддинги для обучения")
        raise ValueError("Embeddings cannot be None")

    logger.info(f"Начало обучения модели KMeans с {n_clusters} кластерами")
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(embeddings)
    global centroids_shape
    centroids_shape = kmeans.cluster_centers_.shape
    global centroids
    centroids = kmeans.cluster_centers_
    logger.info(f"Модель KMeans успешно обучена, центроиды: {centroids_shape}")
    kmeans.__dict__

    return kmeans

In [179]:
def get_ONE_embedding(text, model_name='all-MiniLM-L6-v2'):
    """
    Преобразует текст в эмбеддинг

    Args:
        text (str): Текстовая строка
        model_name (str): Название предобученной модели

    Returns:
        np.array: Нормализованный эмбеддинг
    """
    logger.info(f"Создание эмбеддинга для текста длиной {len(text)} символов")
    model = SentenceTransformer(model_name)
    ONE_embedding = model.encode([text], convert_to_numpy=True)

    # Нормализация L2 (делает длину векторов = 1)
    normalizer = Normalizer(norm='l2')
    ONE_embedding_normalized = normalizer.transform(ONE_embedding)

    # Проверка
    norm_check = np.linalg.norm(ONE_embedding_normalized[0])
    logger.debug(f"Проверка нормализации: норма вектора = {norm_check}")

    return ONE_embedding_normalized


from langdetect import detect

def detect_language(text):
    try:
        lang = detect(text)
        logger.debug(f"Определен язык текста: {lang}")
        return lang
    except Exception as e:
        logger.error(f"Ошибка определения языка: {e}")
        return None

# Путь к папке с файлами
folder_path = "/content/drive/MyDrive/InputData"
logger.info(f"Работа с папкой: {folder_path}")

# Поддерживаемые форматы
supported_extensions = ['.pdf', '.docx', '.txt', '.pptx', '.jpg', '.png', '.jpeg']

# Проходим по всем файлам в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Проверяем расширение файла
    if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in supported_extensions):
        try:
            logger.info(f"Начало обработки файла: {filename}")

            # Извлекаем текст
            text = textract.process(file_path).decode('utf-8')
            logger.debug(f"Извлечен текст из файла {filename}, длина: {len(text)} символов")

            # Определяем язык
            text_lang = detect_language(text)
            logger.info(f"Для файла {filename} определен язык: {text_lang}")

            # # Определение краткого описания текста
            # if text_lang == "ru":
            #     text_description = summarize_text(text, ru_summarizer)
            # elif text_lang == "en":
            #     text_description = summarize_text(text, en_summarizer)
            # else:
            #     text_description = "unknown_language"
            #     logger.warning(f"Неизвестный язык для файла {filename}")

            # logger.info(f"Создано описание для файла {filename}: {text_description}")

            # # Переименование
            # new_path = os.path.join(folder_path, text_description,)
            # shutil.move(file_path, new_path)
            # logger.info(f"Файл {filename} переименован в {text_description}")

        except Exception as e:
            logger.error(f"Ошибка при обработке файла {filename}: {str(e)}")


def move_to_folder(cluster, filename):
    """Перемещает файл в папку соответствующего кластера"""
    directory = "/content/drive/MyDrive/InputData"
    logger.info(f"Перемещение файла {filename} в кластер {cluster}")

    destination_path = f"/content/drive/MyDrive/{cluster}"
    logger.debug(f"Целевая папка: {destination_path}")

    # Создать папку, если её нет
    if not os.path.exists(destination_path):
        logger.debug(f"Создание папки {destination_path}")
        os.makedirs(destination_path)

    begPath = os.path.join(directory, filename)
    try:
        shutil.copy(begPath, destination_path)
        logger.info(f"Файл {filename} успешно скопирован в {destination_path}")
    except Exception as e:
        logger.error(f"Ошибка при копировании файла {filename}: {str(e)}")

Выходные данные были обрезаны до нескольких последних строк (5000).
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue
    await self.process_one()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 499, in process_one
    await dispatch(*args)
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 406, in dispatch_shell
    await result
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 730, in execute_request
    reply_content = await reply_content
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 383, in do_execute
    res = shell.run_cell(
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/zmqshell.py", line 528, in run_cell
    return super().run_cell(*args, **kwargs)
  File "/usr/local/lib/python3.11/dis

In [180]:
texts = []
  # Проходим по всем файлам в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Проверяем расширение файла
    if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in supported_extensions):
        try:
            logger.info(f"Обработка файла: {filename}")
            text = textract.process(file_path).decode('utf-8')
            texts.append(text)
            logger.debug(f"Успешно извлечен текст из файла {filename}, длина текста: {len(text)} символов")
        except Exception as e:
            logger.error(f"Ошибка при обработке файла {filename}: {str(e)}")

if not texts:
    logger.warning("Не найдено ни одного файла с поддерживаемым форматом")

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

In [181]:
'''Очистка текстов'''
for i, text in enumerate(texts):
  space_table = text.maketrans({'\t': None, '\n': None, '\r': None})
  texts[i] = text.translate(space_table)
logger.info(r"Текста очищены от \t, \n, \r")

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

In [182]:
def get_embeddings(texts=[], model_name='all-MiniLM-L6-v2'):
    """
    Преобразует список текстов в эмбеддинги

    Args:
        model_name (str): Название предобученной модели (по умолчанию 'all-MiniLM-L6-v2')

    Returns:
        np.array: Массив эмбеддингов размером (n_texts, embedding_dim)
    """

    model = SentenceTransformer(model_name)
    logger.info(f"Загружена модель для эмбеддингов: {model_name}")

    # Путь к папке с файлами
    folder_path = "/content/drive/MyDrive/InputData"
    logger.info(f"Чтение файлов из папки: {folder_path}")

    # Поддерживаемые форматы
    supported_extensions = ['.pdf', '.docx', '.txt', '.pptx', '.jpg', '.png', '.jpeg']

    logger.info(f"Всего обработано файлов: {len(texts)}")

    embeddings = model.encode(texts, convert_to_numpy=True)
    logger.info(f"Созданы эмбеддинги для {len(texts)} текстов, размерность: {embeddings.shape}")

    # Нормализация L2 (делает длину векторов = 1)
    def normalize_embeddings(embeddings):
      norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
      return (embeddings / norms)

    embeddings_normalized = normalize_embeddings(embeddings)


    logger.debug(f"Выполнена нормализация эмбеддингов")

    logger.debug(f"Искходный эмбеддинг: {embeddings_normalized}")
    logger.debug(f"Нормализованный эмбеддинг: {embeddings}")

    # Проверка
    # norm_check = np.linalg.norm(embeddings_normalized, 1.0)
    norm_check = np.isclose(np.linalg.norm(embeddings_normalized[0]), 1.0)
    logger.debug(f"Проверка нормализации: норма первого вектора = {norm_check}")
    return embeddings

In [183]:
embeddings = get_embeddings(texts)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

In [184]:
kmeans_model = train_cluster_model(embeddings)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

In [185]:
# Путь к папке с файлами
folder_path = "/content/drive/MyDrive/InputData"
logger.info(f"Кластеризация файлов в папке: {folder_path}")

# Поддерживаемые форматы
supported_extensions = ['.pdf', '.docx', '.txt', '.pptx', '.jpg', '.png', '.jpeg']

# Проходим по всем файлам в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Проверяем расширение файла
    if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in supported_extensions):
        try:
            logger.info(f"Обработка файла для кластеризации: {filename}")

            # Извлекаем текст
            text = textract.process(file_path).decode('utf-8')
            logger.debug(f"Извлечен текст из файла {filename}, длина: {len(text)} символов")

            # Находим в какой кластер попадает текст
            cluster = predict_cluster(text, kmeans_model)
            logger.info(f"Файл {filename} отнесен к кластеру {cluster}")

            # Перемещаем файл в папку соответствующего кластера
            move_to_folder(cluster, filename)

        except Exception as e:
            logger.error(f"Ошибка при обработке файла {filename}: {str(e)}")

logger.info("Обработка файлов завершена")

Выходные данные были обрезаны до нескольких последних строк (5000).
    await result
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 730, in execute_request
    reply_content = await reply_content
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py", line 383, in do_execute
    res = shell.run_cell(
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/zmqshell.py", line 528, in run_cell
    return super().run_cell(*args, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2975, in run_cell
    result = self._run_cell(
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3030, in _run_cell
    return runner(coro)
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/async_helpers.py", line 78, in _pseudo_sync_runner
    coro.send(None)
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3257, in run_cell_async
 

In [186]:
closest_texts = find_closest_to_centroids(embeddings, centroids, texts=texts)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

In [187]:
for text in closest_texts:
  lang = detect_language(text)
  text_description = np.array
  if (lang=="en"):
    text_description = np.append(text_description,
                                 summarize_text(text, en_summarizer))
  elif (lang=="ru"):
    text_description = np.append(text_description,
                                 summarize_text(text, ru_summarizer))

--- Logging error ---
Traceback (most recent call last):
  File "<ipython-input-179-183e96f167f6>", line 31, in detect_language
    lang = detect(text)
           ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/langdetect/detector_factory.py", line 130, in detect
    return detector.detect()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/langdetect/detector.py", line 136, in detect
    probabilities = self.get_probabilities()
                    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/langdetect/detector.py", line 143, in get_probabilities
    self._detect_block()
  File "/usr/local/lib/python3.11/dist-packages/langdetect/detector.py", line 150, in _detect_block
    raise LangDetectException(ErrorCode.CantDetectError, 'No features in text.')
langdetect.lang_detect_exception.LangDetectException: No features in text.

During handling of the above exception, another exception occurred:

Traceback (most recent call

In [188]:
text_description[1]

'Acoustic instability appearing in chambers with isothermal or reacting mean'

In [198]:
def rename_folder_cluster(text_description):
  # Переименование
  folder_path = os.PathLike
  for i in range(0, 2):
    old_folder_path = f"/content/drive/MyDrive/{i}"
    new_folder_path = f"/content/drive/MyDrive/{text_description[i]}"
    #folder_path = os.path.join(folder_path, i)
    shutil.move(old_folder_path, new_folder_path)
    logger.info(f"Папка {old_folder_path} переименован в {text_description[i]}")

In [199]:
rename_folder_cluster(text_description)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

In [195]:


def rename_folder_cluster(text_description):
    # Базовый путь
    old_folder_path = "/content/drive/MyDrive/"

    for i in range(0, 2):
        # Полный путь к исходной папке
        old_folder_full_path = os.path.join(old_folder_path, str(i))

        # Проверяем, существует ли папка
        if not os.path.exists(old_folder_full_path):
            logger.error(f"Папка {old_folder_full_path} не существует!")
            continue

        # Новое имя папки (берем из массива описаний)
        new_name = text_description[i]

        # Полный путь к новой папке
        new_folder_full_path = os.path.join(old_folder_path, new_name)

        try:
            # Переименовываем папку
            shutil.move(old_folder_full_path, new_folder_full_path)
            logger.info(f"Папка {old_folder_full_path} переименована в {new_name}")
        except Exception as e:
            logger.error(f"Ошибка при переименовании {old_folder_full_path}: {str(e)}")